In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
filename = "uklon_sample_3_months_dropnan_driver_id.csv"

In [3]:
parse_dates = ['created_at']

In [4]:
df = pd.read_csv(filename, delimiter=",", parse_dates=parse_dates)

/home/temenid/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df.head()

,order_id,ride_id,created_at,pickup_lat,pickup_lon,dropoff_lat,dropoff_lon,user_id,driver_id,ride_distance,...,ooExtraCost,ooInitialCost,ooIdleCost,oapAzureMlCost,oapAzureMlCostBid,oapCostProvider,ooSurgeMultiplier,driver_lat,driver_lon,dtTaxiDriver_tdID
0,88314768,764A34A6-5C92-4A0C-90E8-82BDA5A5F9C1,2017-10-31 23:59:59.143,50.448906,30.386448,50.410267,30.412216,420361.0,568114.0,9.10,...,1.0,115.0,0.0,114.0,1.0,AzureML,1.35,50.437012,30.383680,568114.0
1,88314766,299E8328-8BE6-4097-AF81-4834478B4D51,2017-10-31 23:59:58.577,50.446000,30.576200,50.414604,30.384350,722883.0,553135.0,19.74,...,0.0,146.0,0.0,146.0,0.0,AzureML,1.00,50.449512,30.595537,553135.0
2,88314763,4EDC7FB9-5E34-4132-A2B3-B926F81DB439,2017-10-31 23:59:55.050,50.438610,30.459536,50.439339,30.444427,693323.0,568920.0,11.08,...,-7.0,100.0,0.0,107.0,1.0,AzureML,1.09,50.424972,30.458965,568920.0
3,88314762,D0EE9C9A-CFDD-44F8-906D-7C3A7ED65A1C,2017-10-31 23:59:54.420,50.449707,30.515345,50.459896,30.402002,17563.0,527239.0,9.24,...,0.0,88.0,0.0,88.0,0.0,NaN,1.00,50.455327,30.515555,527239.0
4,88314759,18B41EC0-B379-4621-BF7A-DD881D16E038,2017-10-31 23:59:50.053,50.397000,30.632500,50.416607,30.633417,754011.0,531709.0,3.73,...,0.0,55.0,0.0,NaN,NaN,NaN,1.01,NaN,NaN,NaN


In [6]:
df.columns

Index(['order_id', 'ride_id', 'created_at', 'pickup_lat', 'pickup_lon',
       'dropoff_lat', 'dropoff_lon', 'user_id', 'driver_id', 'ride_distance',
       'ooCancelReason', 'ooSuburbanDistance', 'ooCarSearchTime', 'ooCost',
       'ooExtraCost', 'ooInitialCost', 'ooIdleCost', 'oapAzureMlCost',
       'oapAzureMlCostBid', 'oapCostProvider', 'ooSurgeMultiplier',
       'driver_lat', 'driver_lon', 'dtTaxiDriver_tdID'],
      dtype='object')

In [7]:
df.isnull().sum()

order_id                    0
ride_id                     0
created_at                  0
pickup_lat                565
pickup_lon                565
dropoff_lat               780
dropoff_lon               780
user_id                378190
driver_id                   0
ride_distance               0
ooCancelReason              0
ooSuburbanDistance          0
ooCarSearchTime             0
ooCost                      0
ooExtraCost                 0
ooInitialCost               0
ooIdleCost                  0
oapAzureMlCost        1521212
oapAzureMlCostBid     1521212
oapCostProvider       1350984
ooSurgeMultiplier           0
driver_lat             417486
driver_lon             417486
dtTaxiDriver_tdID      417486
dtype: int64

In [8]:
df.shape

(2815393, 24)

In [9]:
# Remove rows with missing values in column 'driver_id'. The order was not accepted. there was no trip
new_df = df.dropna(subset=['driver_id'],inplace=False)

In [10]:
# leave only uncanceled ride
new_df = new_df.loc[df['ooCancelReason'] == 0]

In [11]:
new_df.shape

(2371694, 24)

In [12]:
# initialy select the following columns
data = new_df[['created_at','ride_distance', 'ooSuburbanDistance', 'ooCarSearchTime', 'ooCost']]

In [13]:
data.head()

,created_at,ride_distance,ooSuburbanDistance,ooCarSearchTime,ooCost
0,2017-10-31 23:59:59.143,9.10,0,13,115.0
1,2017-10-31 23:59:58.577,19.74,0,7,146.0
2,2017-10-31 23:59:55.050,11.08,0,1,100.0
3,2017-10-31 23:59:54.420,9.24,0,69,88.0
4,2017-10-31 23:59:50.053,3.73,0,4,55.0


In [14]:
# count the NaN values in a columns
data.isnull().sum()

created_at            0
ride_distance         0
ooSuburbanDistance    0
ooCarSearchTime       0
ooCost                0
dtype: int64

In [15]:
# some simple feature engineering with date
# as categorical
data['quarter'] = data['created_at'].dt.quarter.astype(str)
data['month'] = data['created_at'].dt.month.astype(str)
data['day'] = data['created_at'].dt.day.astype(str)
data['dayofweek'] = data['created_at'].dt.dayofweek.astype(str)

/home/temenid/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/temenid/.local/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/temenid/.local/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [16]:
# as categorical
data['quarter_int'] = data['created_at'].dt.quarter.astype(int)
data['month_int'] = data['created_at'].dt.month.astype(int)
data['day_int'] = data['created_at'].dt.day.astype(int)
data['dayofweek_int'] = data['created_at'].dt.dayofweek.astype(int)

/home/temenid/.local/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/temenid/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/temenid/.local/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [17]:
data.head()

,created_at,ride_distance,ooSuburbanDistance,ooCarSearchTime,ooCost,quarter,month,day,dayofweek,quarter_int,month_int,day_int,dayofweek_int
0,2017-10-31 23:59:59.143,9.10,0,13,115.0,4,10,31,1,4,10,31,1
1,2017-10-31 23:59:58.577,19.74,0,7,146.0,4,10,31,1,4,10,31,1
2,2017-10-31 23:59:55.050,11.08,0,1,100.0,4,10,31,1,4,10,31,1
3,2017-10-31 23:59:54.420,9.24,0,69,88.0,4,10,31,1,4,10,31,1
4,2017-10-31 23:59:50.053,3.73,0,4,55.0,4,10,31,1,4,10,31,1


In [18]:
data.shape

(2371694, 13)

In [19]:
data.describe()

,ride_distance,ooSuburbanDistance,ooCarSearchTime,ooCost,quarter_int,month_int,day_int,dayofweek_int
count,2.371694e+06,2371694.0,2.371694e+06,2.371694e+06,2.371694e+06,2.371694e+06,2.371694e+06,2.371694e+06
mean,1.104782e+01,0.0,1.909313e+01,1.053575e+02,3.361107e+00,9.048252e+00,1.585837e+01,3.008878e+00
std,7.240361e+00,0.0,3.114566e+01,5.001793e+01,4.803217e-01,8.195334e-01,8.864580e+00,1.954233e+00
min,0.000000e+00,0.0,-3.528000e+03,3.000000e+01,3.000000e+00,8.000000e+00,1.000000e+00,0.000000e+00
25%,5.610000e+00,0.0,5.000000e+00,7.000000e+01,3.000000e+00,8.000000e+00,8.000000e+00,1.000000e+00
50%,9.960000e+00,0.0,9.000000e+00,1.000000e+02,3.000000e+00,9.000000e+00,1.600000e+01,3.000000e+00
75%,1.451000e+01,0.0,2.300000e+01,1.250000e+02,4.000000e+00,1.000000e+01,2.300000e+01,5.000000e+00
max,1.443100e+02,0.0,4.747000e+03,4.062000e+03,4.000000e+00,1.000000e+01,3.100000e+01,6.000000e+00


In [20]:
# In our dataset, we have both categorical and numeric data. select them for different processing

In [21]:
# CATEGORICAL (for the moment only these)
catFeatures = ['quarter','month','day','dayofweek']

In [22]:
# Convert categorical variable into dummy/indicator variables
df_cat = pd.get_dummies(data[catFeatures])

In [23]:
# let's check
df_cat.head()

,quarter_3,quarter_4,month_10,month_8,month_9,day_1,day_10,day_11,day_12,day_13,...,day_7,day_8,day_9,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6
0,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [24]:
# concat dummy/indicator variables to dataframe
data = pd.concat([data, df_cat], axis=1)

In [25]:
data.head()

,created_at,ride_distance,ooSuburbanDistance,ooCarSearchTime,ooCost,quarter,month,day,dayofweek,quarter_int,...,day_7,day_8,day_9,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6
0,2017-10-31 23:59:59.143,9.10,0,13,115.0,4,10,31,1,4,...,0,0,0,0,1,0,0,0,0,0
1,2017-10-31 23:59:58.577,19.74,0,7,146.0,4,10,31,1,4,...,0,0,0,0,1,0,0,0,0,0
2,2017-10-31 23:59:55.050,11.08,0,1,100.0,4,10,31,1,4,...,0,0,0,0,1,0,0,0,0,0
3,2017-10-31 23:59:54.420,9.24,0,69,88.0,4,10,31,1,4,...,0,0,0,0,1,0,0,0,0,0
4,2017-10-31 23:59:50.053,3.73,0,4,55.0,4,10,31,1,4,...,0,0,0,0,1,0,0,0,0,0


In [26]:
# drop useless categorical column (because we have dummy/indicator variables)
data.drop(catFeatures, axis=1,inplace=True)

In [27]:
data.head()

,created_at,ride_distance,ooSuburbanDistance,ooCarSearchTime,ooCost,quarter_int,month_int,day_int,dayofweek_int,quarter_3,...,day_7,day_8,day_9,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6
0,2017-10-31 23:59:59.143,9.10,0,13,115.0,4,10,31,1,0,...,0,0,0,0,1,0,0,0,0,0
1,2017-10-31 23:59:58.577,19.74,0,7,146.0,4,10,31,1,0,...,0,0,0,0,1,0,0,0,0,0
2,2017-10-31 23:59:55.050,11.08,0,1,100.0,4,10,31,1,0,...,0,0,0,0,1,0,0,0,0,0
3,2017-10-31 23:59:54.420,9.24,0,69,88.0,4,10,31,1,0,...,0,0,0,0,1,0,0,0,0,0
4,2017-10-31 23:59:50.053,3.73,0,4,55.0,4,10,31,1,0,...,0,0,0,0,1,0,0,0,0,0


In [28]:
#INTEGER
intFeatures = ['ride_distance','ooSuburbanDistance','ooCarSearchTime','quarter_int','month_int','day_int','dayofweek_int']

In [29]:
# integer features scaling
scaler = MinMaxScaler()
data[intFeatures] = scaler.fit_transform(data[intFeatures])

In [30]:
# split features and target
dfX = data.drop(['created_at','ooCost'], axis=1)
dfY = data['ooCost']

In [31]:
dfX.head()

,ride_distance,ooSuburbanDistance,ooCarSearchTime,quarter_int,month_int,day_int,dayofweek_int,quarter_3,quarter_4,month_10,...,day_7,day_8,day_9,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6
0,0.063059,0.0,0.427915,1.0,1.0,1.0,0.166667,0,1,1,...,0,0,0,0,1,0,0,0,0,0
1,0.136789,0.0,0.427190,1.0,1.0,1.0,0.166667,0,1,1,...,0,0,0,0,1,0,0,0,0,0
2,0.076779,0.0,0.426465,1.0,1.0,1.0,0.166667,0,1,1,...,0,0,0,0,1,0,0,0,0,0
3,0.064029,0.0,0.434683,1.0,1.0,1.0,0.166667,0,1,1,...,0,0,0,0,1,0,0,0,0,0
4,0.025847,0.0,0.426828,1.0,1.0,1.0,0.166667,0,1,1,...,0,0,0,0,1,0,0,0,0,0


In [32]:
dfY.head()

0    115.0
1    146.0
2    100.0
3     88.0
4     55.0
Name: ooCost, dtype: float64

In [33]:
# get features names
fNames = dfX.columns

In [34]:
# Get values
# features
X = dfX[fNames].values
# targets
y = dfY.values

In [35]:
X.shape, y.shape

((2371694, 50), (2371694,))

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

### simple linear regression

In [37]:
# Create linear regression object
regr = linear_model.LinearRegression()

In [38]:
# Train the model using the training sets
regr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [39]:
# Make predictions using the testing set
y_pred = regr.predict(X_test)

In [40]:
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))

Mean squared error: 400.48


In [41]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [42]:
mean_absolute_percentage_error(y_test, y_pred)

11.36418523554799

### RandomForestRegressor

In [43]:
from sklearn.ensemble import RandomForestRegressor

In [50]:
rf = RandomForestRegressor(n_estimators=50, random_state=1)
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
           oob_score=False, random_state=1, verbose=0, warm_start=False)

In [51]:
# Make predictions using the testing set
y_pred = rf.predict(X_test)

In [52]:
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))

Mean squared error: 452.66


In [53]:
mean_absolute_percentage_error(y_test, y_pred)

12.486953663742883